# Environment setting

In [9]:
# move to source directory
%pwd
%cd ../src
%pwd
%ls

/home/yh/git/kaggle_ieee-fraud-detection/src
__pycache__/    feature_base.py      modelapi_factory.py*  transformer.py*
config/         feature_factory.py*  models/               utils/
configure.py*   features/            tests/                visualization/
experiment.py*  main.py*             trainer_factory.py*


In [10]:
# import major modules
import pandas as pd
import numpy as np
import sys
from pathlib import Path
sys.stdout.flush()

In [11]:
# major dirs
ROOTDIR = Path('/home/yh/git/kaggle_ieee-fraud-detection/').resolve()
DATA_DIR = ROOTDIR / 'data/processed'
RAW_DIR = ROOTDIR / 'data/raw'

# Read data

In [12]:
train = pd.read_pickle(DATA_DIR / 'features_train.pkl')
test = pd.read_pickle(DATA_DIR / 'features_test.pkl')

train.shape, test.shape

((590540, 266), (506691, 265))

In [14]:
train.reset_index(inplace=True)
train.set_index('TransactionID', drop=True, inplace=True)
